In [267]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [268]:
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import re
from datetime import datetime
import itertools

In [269]:
def inisialize_df():
    global df, features
    
    # Create data frame to add the data collected 
    df = pd.DataFrame()
    
    # features name
    features = { 0:'price',1:'color', 2:'fuel_type', 3:'car_origin',4:'driving_license',
                        5:'gear_type', 6:'glass', 7:'Motor_power', 8:'Vehicle_counter',
                        9:'passengers_numbers', 10:'Payment_method', 11:'Offered',
                        12:'Former_owners', 13:'additional_information',
                        14:'insurance_third_party', 15:'insurance_body',
                        16:'insurance_total', 17:'start_date', 18:'end_date'}

In [270]:
def get_price(price):
    price = str(price.text)
    for subitem in price.split():
        if(subitem.isdigit()):
            price = subitem
            break
    return price

In [271]:
def get_description(description_value):

    sample = []

    for row in description_value:  
        feature_value = ''
        columns = row.findAll('td')
        
        if(columns != []):
            feature_value = columns[1].text
            sample.append(feature_value)
    return sample

In [272]:
def get_additional(additional_value):
    
    sample = []
    additional_value = additional_value.findAll('li')

    for row in additional_value:  
        feature_value = ''
        feature_value += row.text            
        sample.append(feature_value)
    sample = ','.join(map(str, sample))

    return sample

In [273]:
def get_insurance(insurance_values):
    n = []
    for row in insurance_values:
        subitem = str(row.text).split()
        for i in range(len(subitem)):
            if(subitem[i].isdigit()):
                n += subitem
    return n

In [274]:
def get_post_info(create_post):
    
    date_cleaned = []

    feature_value = str(create_post[2].text).split()
    date_dirty = [feature_value[5], feature_value[8]]
    
    for i in range(2):
        match = re.search(r'\d{4}-\d{2}-\d{2}', date_dirty[i])
        date = datetime.strptime(match.group(), '%Y-%m-%d').date()
        date_cleaned.append(str(date))
    
    return date_cleaned

In [275]:
inisialize_df()
count = 0

In [276]:
for p in Path('data/').glob('*.txt'):
    with p.open(encoding='utf-8') as fp:
        soup = BeautifulSoup(fp, "html.parser")
        
        price = soup.find('h5', class_='post-price')
        price = [get_price(price)]
        
        description_value = soup.findAll('tr', class_='list-row', limit=12)
        description_value = get_description(description_value)
        
        addition_values = soup.find('td', class_='list-additions')
        addition_values = [get_additional(addition_values)]
        
        insurance_info = soup.find('div', class_='row').findAll('td', attrs={'class': None, 'colspan':None})
        insurance_info = get_insurance(insurance_info)
        
        
        create_post = soup.findAll('table', class_='create_post')
        create_post = get_post_info(create_post)
        
        sample = itertools.chain(price,description_value,addition_values,insurance_info, create_post)
        sample = list(sample)
        df[count] = sample
        count += 1

IndexError: list index out of range

In [277]:
df.rename(index=features)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
price,100000,60000,43500,5500,54000,205000,65000,230000,59000,84000,75000,39000,73000,30000
color,أبيض عاجي,سكني,سكني,بيج,فضي,أسود,بترولي,أبيض,أسود ميتالك,أبيض,أبيض عاجي,فضي,أبيض,ذهبي
fuel_type,بنزين,ديزل,بنزين,بنزين,بنزين,ديزل,بنزين,بنزين,بنزين,بنزين,ديزل,بنزين,بنزين,بنزين
car_origin,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي,خصوصي
driving_license,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية,فلسطينية
gear_type,اوتوماتيك,نصف اوتوماتيك,اوتوماتيك,عادي,اوتوماتيك,اوتوماتيك,عادي,اوتوماتيك,اوتوماتيك,عادي,عادي,عادي,اوتوماتيك,اوتوماتيك
glass,الكتروني,الكتروني,الكتروني,يدوي,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني,الكتروني
Motor_power,2000,2500,1600,906,1200,2000,1400,2000,1600,4500,2500,1400,1200,1600
Vehicle_counter,75000,130000,نقدا فقط,شغال,38000,5+1,١٢٣٤٥٦,50000,نقدا فقط,56000,460000,4+1,45000,70000
passengers_numbers,4+1,7+1,للبيع فقط,4+1,4+1,نقدا فقط,٤+١,4+1,للبيع فقط,8,7+1,نقدا فقط,4+1,4+1
